In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer

# Loading Dataset
meta_data = pd.read_csv("HAM10000_metadata.csv")

# Getting Labels
# all_labels = meta_data['dx']
labels = meta_data['dx'].isin(['bcc', 'mel', 'akiec'])
meta_attributes = meta_data[["dx", "dx_type", "age", "sex", "localization"]]

# removing missing values
imputer = SimpleImputer(strategy='most_frequent')
for col in meta_attributes.columns:
    # print(f"{col=}")
    meta_attributes.loc[:, col] = imputer.fit_transform(
        meta_attributes[[col]]).ravel()


In [38]:
import cv2
import numpy as np

def find_and_crop_mole(image_path, threshold):
    global mole_region
    global original_image
    global cropped_mole
    global gray_image
    global blurred_image

    original_image = cv2.imread(image_path)

    # grayscale
    gray_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)

    # Gaussian blur
    blurred_image = cv2.GaussianBlur(gray_image, (1, 1), 0)

    _, binary_image = cv2.threshold(
        blurred_image, threshold, 255, cv2.THRESH_BINARY)

    contours, _ = cv2.findContours(
        binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # with the largest contour
    area = 0
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        area = len(largest_contour)
        # print(f"{area=}")
        # mask for the mole
        mask = np.zeros_like(original_image)
        cv2.drawContours(mask, [largest_contour], 0,
                         (255, 255, 255), thickness=cv2.FILLED)

        mole_region = cv2.bitwise_and(original_image, mask)

        x, y, w, h = cv2.boundingRect(largest_contour)
        # print(f"{x, y, w, h=}")

        cropped_mole = original_image[y:y+h, x:x+w]
        
        # cv2.imwrite("cropped_mole.jpg", cropped_mole)
    else:
        print("No mole found in the image.")
    return area

threshold = 50
print(f"Initial Threshold: {threshold}")
while (find_and_crop_mole("HAM10000_images_part_1\ISIC_0024306.jpg", threshold) < 1000):
    threshold += 1
    # cv2.imshow("Mole Region", mole_region)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    cv2.imwrite(f"tmp/cropped_mole_{threshold}.jpg", mole_region)
print(f"Final Threshold: {threshold}")
cv2.imwrite(f"tmp/cropped_mole_{threshold}.jpg", mole_region)



cv2.imshow("Original Image", original_image)
# cv2.imshow("Gray Image", gray_image)
# cv2.imshow("Blurred Image", blurred_image)
cv2.imshow("Mole Region", mole_region)
cv2.imshow("Cropped Image (Not Working Properly)", cropped_mole)
cv2.waitKey(0)
cv2.destroyAllWindows()


Initial Threshold: 50
Final Threshold: 110
